# Try out gradio

**Load and try the model**

In [16]:
# Prerequisites
from tabulate import tabulate
from transformers import pipeline
import json
import pandas as pd

# Load the model and create a pipeline for zero-shot classification (1min loading + classifying with 89 labels)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# get candidate labels
with open("packing_label_structure.json", "r") as file:
    candidate_labels = json.load(file)
keys_list = list(candidate_labels.keys())

# Load test data (in list of dictionaries)
with open("test_data.json", "r") as file:
    packing_data = json.load(file)
# Extract all trip descriptions and trip_types
trip_descriptions = [trip['description'] for trip in packing_data]
trip_types = [trip['trip_types'] for trip in packing_data]

# Access the first trip description
first_trip = trip_descriptions[0]
# Get the packing list for the secondfirst trip
first_trip_type = trip_types[0]

print(f"First trip: {first_trip} \n")
print(f"Trip type: {first_trip_type}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


First trip: I am planning a trip to Greece with my boyfriend, where we will visit two islands. We have booked an apartment on each island for a few days and plan to spend most of our time relaxing. Our main goals are to enjoy the beach, try delicious local food, and possibly go on a hike—if it’s not too hot. We will be relying solely on public transport. We’re in our late 20s and traveling from the Netherlands. 

Trip type: ['beach vacation', ['swimming', 'going to the beach', 'relaxing', 'hiking'], 'warm destination / summer', 'lightweight (but comfortable)', 'casual', 'indoor', 'no vehicle', 'no special conditions', '7 days']


In [37]:
# Create an empty DataFrame with specified columns
df = pd.DataFrame(columns=['superclass', 'pred_class'])
cutoff = 0.5  # used to choose which activities are relevant

# fill DataFrame
for i, key in enumerate(keys_list):
    # Run the classification (ca 30 seconds classifying)
    if key == 'activities':
        result = classifier(first_trip, candidate_labels[key], multi_label=True)
        indices = [i for i, score in enumerate(result['scores']) if score > cutoff]
        classes = [result['labels'][i] for i in indices]
    else:
        result = classifier(first_trip, candidate_labels[key])
        classes = result["labels"][0]
    print(result)
    print(classes)
    print(i)
    df.loc[i] = [key, classes]

df['true_class'] = first_trip_type

{'sequence': 'I am planning a trip to Greece with my boyfriend, where we will visit two islands. We have booked an apartment on each island for a few days and plan to spend most of our time relaxing. Our main goals are to enjoy the beach, try delicious local food, and possibly go on a hike—if it’s not too hot. We will be relying solely on public transport. We’re in our late 20s and traveling from the Netherlands.', 'labels': ['beach vacation', 'micro-adventure / weekend trip', 'nature escape', 'digital nomad trip', 'cultural exploration', 'yoga / wellness retreat', 'festival trip', 'long-distance hike / thru-hike', 'hut trek (summer)', 'city trip', 'road trip (car/camper)', 'ski tour / skitour', 'camping trip (campground)', 'snowboard / splitboard trip', 'camping trip (wild camping)', 'hut trek (winter)'], 'scores': [0.37631064653396606, 0.35016775131225586, 0.13397355377674103, 0.031636204570531845, 0.031270742416381836, 0.012846449390053749, 0.012699575163424015, 0.009526746347546577

In [40]:
pd.set_option('display.width', 1000) 
pd.set_option('display.max_columns', None)
print(df)

           Superclass                              pred_class                                        true_class
0       activity_type                          beach vacation                                    beach vacation
1          activities  [going to the beach, relaxing, hiking]  [swimming, going to the beach, relaxing, hiking]
2   climate_or_season               warm destination / summer                         warm destination / summer
3    style_or_comfort                              minimalist                     lightweight (but comfortable)
4          dress_code                                  casual                                            casual
5       accommodation                    huts with half board                                            indoor
6      transportation                                 vehicle                                        no vehicle
7  special_conditions               off-grid / no electricity                             no special con

In [42]:
df['same'] = df['pred_class'] == df['true_class']
print(df)

           Superclass                              pred_class                                        true_class   same
0       activity_type                          beach vacation                                    beach vacation   True
1          activities  [going to the beach, relaxing, hiking]  [swimming, going to the beach, relaxing, hiking]  False
2   climate_or_season               warm destination / summer                         warm destination / summer   True
3    style_or_comfort                              minimalist                     lightweight (but comfortable)  False
4          dress_code                                  casual                                            casual   True
5       accommodation                    huts with half board                                            indoor  False
6      transportation                                 vehicle                                        no vehicle  False
7  special_conditions               off-grid / n

In [62]:
# accuracy excluding activities
correct = sum(df.loc[df.index != 1, 'same'])
total = len(df['same'])
accuracy = correct/total
print("Accuracy (excluding activities):", accuracy)

Accuracy (excluding activities): 0.3333333333333333


In [64]:
pred_class = df.loc[df.index == 1, 'pred_class'].iloc[0]
true_class = df.loc[df.index == 1, 'true_class'].iloc[0]
correct = [label for label in pred_class if label in true_class]

num_correct = len(correct)
correct_perc = num_correct/len(true_class)

num_pred = len(pred_class)
wrong_perc = (num_pred - num_correct)/num_pred

print("Percentage of true classes that were identified:", correct_perc)
print("Percentage of predicted classes that were wrong:", wrong_perc)

Percentage of true classes that were identified: 0.75
Percentage of predicted classes that were wrong: 0.0


Now do the same for all trips

**Use gradio for user input**

In [66]:
# use model with gradio
from transformers import pipeline
import gradio as gr

# make a function for what I am doing
def classify(text):
    df = pd.DataFrame(columns=['Superclass', 'class'])
    for i, key in enumerate(keys_list):
        # Run the classification (ca 30 seconds classifying)
        if key == 'activities':
            result = classifier(text, candidate_labels[key], multi_label=True)
            classes = [result['labels'][i] for i in indices]
        else:
            result = classifier(text, candidate_labels[key])
            classes = result["labels"][0]
        print(i)
        df.loc[i] = [key, classes]

    return df

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs="dataframe",
    title="Zero-Shot Classification",
    description="Enter a text describing your trip",
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://aa06d5d85ffadaa92b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8


Use model with gradio

In [4]:
# Define the Gradio interface
def classify(text):
    return classifier(text, class_labels)

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs="json",
    title="Zero-Shot Classification",
    description="Enter a text describing your trip",
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://0f70ba5369d721cf8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
